In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers,Model

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [3]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download(['averaged_perceptron_tagger','stopwords','punkt','movie_reviews'])
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [4]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [105]:
df=pd.read_excel("/content/drive/MyDrive/regresser/behaviour_simulation_test_time.xlsx")
df.head()

,id,date,content,username,media,inferred company
0,1,2021-11-08 00:40:49,"Andres, a Pharmacy Manager at Store 4669 in Pi...",WalmartWorld,[Photo(previewUrl='https://pbs.twimg.com/media...,walmart
1,2,2021-05-09 07:51:21,Happy #EuropeDay! We are always stronger by wo...,SimonHarrisTD,[Photo(previewUrl='https://pbs.twimg.com/media...,td
2,3,2022-11-22 05:30:05,"Election Results, BVAS Report Established Over...",IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
3,4,2021-09-22 21:52:08,American Olympic swimmer Elizabeth Beisel will...,CBCOlympics,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
4,5,2022-04-15 13:00:27,No need to hunt ‘em or hope to find ‘em in a b...,TimHortonsUS,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons


In [75]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
curr_time = pd.to_datetime("now")
df['diff_days']=curr_time-df['date']
df['diff_days']=df['diff_days']//np.timedelta64(1,'D')

In [77]:
y=df.iloc[:,-1].values
y.shape

(10000,)

In [78]:
def extract_hashtags(x):
    x=x.strip()
    # print(x)
    hash_tags=[]
    x=x.split()
    for i in x:
        if len(i)>0:
            if i[0]=="#":
                hash_tags.append(i)
    k=",".join(hash_tags)
    k.replace("#","")
    return k
    # hashtags=[i for i in x.split(" ") if (len(i)>0) & (i[0]=="#")]
    # k=",".join(hashtags)
    # k.replace("#","")
    # # print(k)
    # # print(hashtags)
    # return k

In [79]:
df['hashtags']=df['content'].map(lambda x: extract_hashtags(x))
df.head()

,id,date,content,username,media,inferred company,diff_days,hashtags
0,1,2021-11-08 00:40:49,"Andres, a Pharmacy Manager at Store 4669 in Pi...",WalmartWorld,[Photo(previewUrl='https://pbs.twimg.com/media...,walmart,767,
1,2,2021-05-09 07:51:21,Happy #EuropeDay! We are always stronger by wo...,SimonHarrisTD,[Photo(previewUrl='https://pbs.twimg.com/media...,td,950,"#EuropeDay!,#HorizonEurope"
2,3,2022-11-22 05:30:05,"Election Results, BVAS Report Established Over...",IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,388,
3,4,2021-09-22 21:52:08,American Olympic swimmer Elizabeth Beisel will...,CBCOlympics,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,813,
4,5,2022-04-15 13:00:27,No need to hunt ‘em or hope to find ‘em in a b...,TimHortonsUS,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,609,


In [80]:
!pip install wordninja
import wordninja

In [81]:
def cleaning_process(text1) :
    text = text1
    #print(text)
    if type(text) == str :
        text = re.sub(r"http\S+", "", text) # removing url

        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
        #print(emoji_pattern.sub(r'', text)) # no emoji
        text = emoji_pattern.sub(r'', text)

        nopunc = [char for char in text if char not in string.punctuation] # removing punctuation
        text = ''.join(nopunc)

        text = ''.join([i for i in text if not i.isdigit()]) # removing digits

        text = text.lower()

        tokens = nltk.word_tokenize(text)
        tokens = [w for w in tokens if w not in stopwords.words('english')]
        #tokens = nltk.word_tokenize(text)

        #lematizer = WordNetLemmatizer()
        #ps = PorterStemmer()
        #lem_text = [lematizer.lemmatize(i) for i in tokens]
        #stem_text = [ps.stem(i) for i in lem_text]
        text = ' '.join(tokens)

        return text

    return None

In [82]:
df['clean_caption']=df['content'].apply(cleaning_process)
df['cleaned_hastags'] = df['hashtags'].apply(cleaning_process)

In [83]:
df['clean_caption'] = df['clean_caption'].apply(lambda x : ' '.join(wordninja.split(x)))
df['cleaned_hastags'] = df['cleaned_hastags'].apply(lambda x : ' '.join(wordninja.split(x)))

In [84]:
X1 = df['cleaned_hastags']
X1= df['cleaned_hastags'].fillna('')
tfidf = TfidfVectorizer(lowercase=False)
# tfidf = CountVectorizer()
text_vect = tfidf.fit_transform(X1)

In [85]:
words_idf_dict = dict()
words_list = tfidf.get_feature_names_out()
idf_list = tfidf.idf_
words_idf_dict = {words_list[i]:idf_list[i] for i in range(len(words_list))}

In [86]:
def get_idf_scores(text) :
    #print(text)
    t2 = nltk.word_tokenize(text)


    idf_list = [words_idf_dict[w] for w in t2 if len(w) > 1]
    avg_idf = np.mean(idf_list)

    return avg_idf

In [87]:
df['avg_idf'] = X1.apply(get_idf_scores)

In [88]:
df['length_hashtags'] = (df['hashtags'].fillna('')).apply(lambda x : x.count('#')) # no of topics
df['length_caption'] = df['clean_caption'].apply(lambda x : len(x.split(" "))) # no of words
# text polarity
from textblob import TextBlob
df['text_polarity'] = df['clean_caption'].apply(lambda x : TextBlob(x).sentiment[0])
df['text_subjectivity'] = df['clean_caption'].apply(lambda x : TextBlob(x).sentiment[1])
df.head()

,id,date,content,username,media,inferred company,diff_days,hashtags,clean_caption,cleaned_hastags,avg_idf,length_hashtags,length_caption,text_polarity,text_subjectivity
0,1,2021-11-08 00:40:49,"Andres, a Pharmacy Manager at Store 4669 in Pi...",WalmartWorld,[Photo(previewUrl='https://pbs.twimg.com/media...,walmart,767,,andres pharmacy manager store pinellas park fl...,,NaN,0,31,0.111111,0.250000
1,2,2021-05-09 07:51:21,Happy #EuropeDay! We are always stronger by wo...,SimonHarrisTD,[Photo(previewUrl='https://pbs.twimg.com/media...,td,950,"#EuropeDay!,#HorizonEurope",happy europe day always stronger working toget...,europe day horizon europe,7.785588,2,32,0.287500,0.500000
2,3,2022-11-22 05:30:05,"Election Results, BVAS Report Established Over...",IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,388,,election results b vas report established over...,,NaN,0,15,0.000000,0.000000
3,4,2021-09-22 21:52:08,American Olympic swimmer Elizabeth Beisel will...,CBCOlympics,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,813,,american olympic swimmer elizabeth be is el at...,,NaN,0,32,0.012500,0.333333
4,5,2022-04-15 13:00:27,No need to hunt ‘em or hope to find ‘em in a b...,TimHortonsUS,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,609,,need hunt em hope find em basket head nearest ...,,NaN,0,35,0.138312,0.415801


In [89]:
df=df[["diff_days","inferred company","clean_caption","avg_idf","length_hashtags","length_caption","text_subjectivity",'text_polarity']]

In [91]:
tfidf_ic = tfidf.fit_transform(df['inferred company'])
tfidf_tokens = tfidf.get_feature_names_out()
df_tfidfvect1 = pd.DataFrame(data = tfidf_ic.toarray(),columns = tfidf_tokens)

In [92]:
df_tfidfvect1.head()

,aa,aaa,aarp,accenture,acer,airlines,airtel,airways,alaska,alcoa,...,wells,wild,williams,wings,wireless,wolseley,wwf,yahoo,ymca,zumba
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
df.drop(['inferred company', 'clean_caption'],inplace =True, axis=1)

In [94]:
df['avg_idf'] = df['avg_idf'].fillna(0)

In [95]:
result = df

In [ ]:
model=tf.keras.Sequential()

for i in range(0,5):
    if i==0:
        model.add(layers.Input(shape=(284,),batch_size=32))
        model.add(layers.Dense(64,activation='relu'))
    elif i%4==0:
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.2))
    else:
        model.add(layers.Dense(128,activation='relu'))

model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(1,activation='linear'))
model.load_weights('/kaggle/working/model.hdf5')
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),loss="mean_squared_error",metrics=['mae','accuracy'])
model.summary()

In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger

# Tensorboard is a tool to monitor training.
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/regresser/tensorboard')

# Callback to save model after every epo/ch.
cp_callback = ModelCheckpoint(filepath='/content/drive/MyDrive/regresser/model.hdf5',
                             monitor='loss',
                             save_freq='epoch',
                             verbose=0,
                             period=1,
                             save_best_only=False,
                             save_weights_only=True)

# Keeps monitoring training and terminates it if model starts to overfit or value of specified monitoring metric is stationary.
es_callback = EarlyStopping(monitor='accuracy',
                            mode='min',
                            verbose=0,
                            patience=500)

# Dumps train and validation accuracy and loss to a csv file.
csv_logger = CSVLogger(filename='/content/drive/MyDrive/regresser/train_logs.csv',
                       separator=',',
                       append=False)

callbacks = [tensorboard, cp_callback, es_callback, csv_logger]

In [ ]:
result.drop(['inferred company', 'clean_caption'], inplace=True,axis=1)
result.head()

,aa,aaa,aarp,accenture,acer,aid,airasia,airlines,airtel,airways,...,wwf,yahoo,ymca,zumba,diff_days,avg_idf,length_hashtags,length_caption,text_subjectivity,text_polarity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1097,8.159732,1,22,0.325000,0.175000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1993,0.000000,0,7,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1171,0.000000,0,12,0.100000,-0.100000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1169,0.000000,0,14,0.900000,0.500000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1882,8.533557,1,23,0.088889,0.133333


In [ ]:
k = result.iloc[:,1:].values


In [99]:
p =model.predict(k)

313/313 [==============================] - 1s 1ms/step


In [102]:
u =(np.ceil(abs(p))).astype(int).tolist()

In [108]:
new = list(np.concatenate(u))


In [106]:
df['likes'] = new

In [107]:
df.to_excel('/content/pred2.xlsx')